In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf

emo_df=pd.read_csv("C:/KM/Datasets/Emotions Data/text.csv")
emo_df

,Unnamed: 0,text,label
0,0,i just feel really helpless and heavy hearted,4
1,1,ive enjoyed being able to slouch about relax a...,0
2,2,i gave up my internship with the dmrg and am f...,4
3,3,i dont know i feel so lost,0
4,4,i am a kindergarten teacher and i am thoroughl...,4
...,...,...,...
416804,416804,i feel like telling these horny devils to find...,2
416805,416805,i began to realize that when i was feeling agi...,3
416806,416806,i feel very curious be why previous early dawn...,5
416807,416807,i feel that becuase of the tyranical nature of...,3


In [2]:
emo_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 416809 entries, 0 to 416808
Data columns (total 3 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   Unnamed: 0  416809 non-null  int64 
 1   text        416809 non-null  object
 2   label       416809 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 9.5+ MB


In [3]:
emo_df.drop('Unnamed: 0',inplace=True,axis=1)
emo_df

,text,label
0,i just feel really helpless and heavy hearted,4
1,ive enjoyed being able to slouch about relax a...,0
2,i gave up my internship with the dmrg and am f...,4
3,i dont know i feel so lost,0
4,i am a kindergarten teacher and i am thoroughl...,4
...,...,...
416804,i feel like telling these horny devils to find...,2
416805,i began to realize that when i was feeling agi...,3
416806,i feel very curious be why previous early dawn...,5
416807,i feel that becuase of the tyranical nature of...,3


In [4]:
emo_df['label'].unique()

array([4, 0, 2, 1, 5, 3])

In [5]:
from sklearn.model_selection import train_test_split
train_set,val_set=train_test_split(emo_df,test_size=0.25,random_state=42)

In [6]:
train_inputs=train_set['text'].to_numpy()
val_inputs=val_set['text'].to_numpy()
train_target=train_set['label'].to_numpy()
val_target=val_set['label'].to_numpy()

In [7]:
from tensorflow.keras.layers import TextVectorization,Embedding

In [8]:
text_vectorizer=TextVectorization(max_tokens=None,split='whitespace',standardize='lower_and_strip_punctuation',output_mode='int',output_sequence_length=None,ngrams=None)

In [9]:
round(sum([len(i.split()) for i in train_inputs])/len(train_inputs)) # detrmining output length 

19

In [10]:
max_vocab=10000
max_length=20

In [11]:
text_vectorizer=TextVectorization(max_tokens=max_vocab,output_sequence_length=max_length)

In [12]:
text_vectorizer.adapt(train_inputs)

In [13]:
embed=Embedding(input_dim=max_vocab,output_dim=128)

In [14]:
from tensorflow.keras.layers import Input,LSTM,GRU,Bidirectional,GlobalAveragePooling1D,Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import CategoricalCrossentropy,SparseCategoricalCrossentropy
from tensorflow.keras.models import Model

In [15]:
inputs=Input(shape=(1,),dtype=tf.string)
x=text_vectorizer(inputs)
x=embed(x)
x=GRU(48,activation='tanh')(x)
outputs=Dense(6,activation='softmax')(x)
model_1=Model(inputs,outputs)
model_1.compile(loss=CategoricalCrossentropy(),optimizer=Adam(),metrics=['accuracy'])
model_1.fit(train_inputs,tf.one_hot(train_target,depth=6),epochs=5,validation_data=(val_inputs,tf.one_hot(val_target,depth=6)))

Epoch 1/5
9769/9769 ━━━━━━━━━━━━━━━━━━━━ 70s 7ms/step - accuracy: 0.7749 - loss: 0.5394 - val_accuracy: 0.8717 - val_loss: 0.2697
Epoch 2/5
9769/9769 ━━━━━━━━━━━━━━━━━━━━ 77s 8ms/step - accuracy: 0.8793 - loss: 0.2547 - val_accuracy: 0.8721 - val_loss: 0.2696
Epoch 3/5
9769/9769 ━━━━━━━━━━━━━━━━━━━━ 68s 7ms/step - accuracy: 0.8896 - loss: 0.2318 - val_accuracy: 0.8683 - val_loss: 0.2847
Epoch 4/5
9769/9769 ━━━━━━━━━━━━━━━━━━━━ 62s 6ms/step - accuracy: 0.9006 - loss: 0.2058 - val_accuracy: 0.8623 - val_loss: 0.3036
Epoch 5/5
9769/9769 ━━━━━━━━━━━━━━━━━━━━ 66s 7ms/step - accuracy: 0.9106 - loss: 0.1822 - val_accuracy: 0.8602 - val_loss: 0.3224


In [16]:
inputs=Input(shape=(1,),dtype=tf.string)
x=text_vectorizer(inputs)
x=embed(x)
x=LSTM(48,activation='tanh')(x)
outputs=Dense(6,activation='softmax')(x)
model_2=Model(inputs,outputs)
model_2.compile(loss=CategoricalCrossentropy(),optimizer=Adam(),metrics=['accuracy','precision','recall'])
model_2.fit(train_inputs,tf.one_hot(train_target,depth=6),epochs=5,validation_data=(val_inputs,tf.one_hot(val_target,depth=6)))

Epoch 1/5
9769/9769 ━━━━━━━━━━━━━━━━━━━━ 66s 7ms/step - accuracy: 0.8543 - loss: 0.3572 - precision: 0.9174 - recall: 0.8016 - val_accuracy: 0.8660 - val_loss: 0.2980 - val_precision: 0.9044 - val_recall: 0.8446
Epoch 2/5
9769/9769 ━━━━━━━━━━━━━━━━━━━━ 63s 6ms/step - accuracy: 0.9042 - loss: 0.2001 - precision: 0.9306 - recall: 0.8842 - val_accuracy: 0.8635 - val_loss: 0.3108 - val_precision: 0.8997 - val_recall: 0.8449
Epoch 3/5
9769/9769 ━━━━━━━━━━━━━━━━━━━━ 63s 6ms/step - accuracy: 0.9135 - loss: 0.1768 - precision: 0.9323 - recall: 0.8983 - val_accuracy: 0.8604 - val_loss: 0.3326 - val_precision: 0.8890 - val_recall: 0.8467
Epoch 4/5
9769/9769 ━━━━━━━━━━━━━━━━━━━━ 115s 12ms/step - accuracy: 0.9199 - loss: 0.1593 - precision: 0.9342 - recall: 0.9082 - val_accuracy: 0.8586 - val_loss: 0.3653 - val_precision: 0.8828 - val_recall: 0.8479
Epoch 5/5
9769/9769 ━━━━━━━━━━━━━━━━━━━━ 64s 7ms/step - accuracy: 0.9257 - loss: 0.1449 - precision: 0.9372 - recall: 0.9164 - val_accuracy: 0.8539 - 

In [17]:
inputs=Input(shape=(1,),dtype=tf.string)
x=text_vectorizer(inputs)
x=embed(x)
x=Bidirectional(GRU(32,activation='tanh'))(x)
outputs=Dense(6,activation='softmax')(x)
model_3=Model(inputs,outputs)
model_3.compile(loss=CategoricalCrossentropy(),optimizer=Adam(),metrics=['accuracy','precision','recall'])
model_3.fit(train_inputs,tf.one_hot(train_target,depth=6),epochs=5,validation_data=(val_inputs,tf.one_hot(val_target,depth=6)))

Epoch 1/5
9769/9769 ━━━━━━━━━━━━━━━━━━━━ 94s 9ms/step - accuracy: 0.8636 - loss: 0.3312 - precision: 0.9154 - recall: 0.8238 - val_accuracy: 0.8602 - val_loss: 0.3350 - val_precision: 0.8901 - val_recall: 0.8463
Epoch 2/5
9769/9769 ━━━━━━━━━━━━━━━━━━━━ 82s 8ms/step - accuracy: 0.9159 - loss: 0.1724 - precision: 0.9321 - recall: 0.9027 - val_accuracy: 0.8554 - val_loss: 0.3545 - val_precision: 0.8795 - val_recall: 0.8440
Epoch 3/5
9769/9769 ━━━━━━━━━━━━━━━━━━━━ 89s 9ms/step - accuracy: 0.9240 - loss: 0.1521 - precision: 0.9356 - recall: 0.9140 - val_accuracy: 0.8562 - val_loss: 0.3888 - val_precision: 0.8751 - val_recall: 0.8481
Epoch 4/5
9769/9769 ━━━━━━━━━━━━━━━━━━━━ 82s 8ms/step - accuracy: 0.9302 - loss: 0.1351 - precision: 0.9389 - recall: 0.9226 - val_accuracy: 0.8526 - val_loss: 0.4234 - val_precision: 0.8700 - val_recall: 0.8457
Epoch 5/5
9769/9769 ━━━━━━━━━━━━━━━━━━━━ 87s 9ms/step - accuracy: 0.9344 - loss: 0.1244 - precision: 0.9415 - recall: 0.9286 - val_accuracy: 0.8504 - va

In [18]:
inputs=Input(shape=(1,),dtype=tf.string)
x=text_vectorizer(inputs)
x=embed(x)
x=Bidirectional(LSTM(32,activation='tanh'))(x)
outputs=Dense(6,activation='softmax')(x)
model_4=Model(inputs,outputs)
model_4.compile(loss=CategoricalCrossentropy(),optimizer=Adam(),metrics=['accuracy','precision','recall'])
model_4.fit(train_inputs,tf.one_hot(train_target,depth=6),epochs=5,validation_data=(val_inputs,tf.one_hot(val_target,depth=6)))

Epoch 1/5
9769/9769 ━━━━━━━━━━━━━━━━━━━━ 80s 8ms/step - accuracy: 0.8697 - loss: 0.3154 - precision: 0.9171 - recall: 0.8310 - val_accuracy: 0.8542 - val_loss: 0.3566 - val_precision: 0.8833 - val_recall: 0.8412
Epoch 2/5
9769/9769 ━━━━━━━━━━━━━━━━━━━━ 80s 8ms/step - accuracy: 0.9186 - loss: 0.1649 - precision: 0.9343 - recall: 0.9061 - val_accuracy: 0.8540 - val_loss: 0.3767 - val_precision: 0.8746 - val_recall: 0.8444
Epoch 3/5
9769/9769 ━━━━━━━━━━━━━━━━━━━━ 82s 8ms/step - accuracy: 0.9272 - loss: 0.1425 - precision: 0.9372 - recall: 0.9184 - val_accuracy: 0.8495 - val_loss: 0.4240 - val_precision: 0.8680 - val_recall: 0.8420
Epoch 4/5
9769/9769 ━━━━━━━━━━━━━━━━━━━━ 84s 9ms/step - accuracy: 0.9337 - loss: 0.1273 - precision: 0.9412 - recall: 0.9271 - val_accuracy: 0.8499 - val_loss: 0.4539 - val_precision: 0.8647 - val_recall: 0.8441
Epoch 5/5
9769/9769 ━━━━━━━━━━━━━━━━━━━━ 80s 8ms/step - accuracy: 0.9367 - loss: 0.1178 - precision: 0.9424 - recall: 0.9319 - val_accuracy: 0.8453 - va

In [19]:
model_1.evaluate(val_inputs,tf.one_hot(val_target,depth=6))

3257/3257 ━━━━━━━━━━━━━━━━━━━━ 7s 2ms/step - accuracy: 0.8530 - loss: 0.4232


[0.42521438002586365, 0.8516837358474731]

In [21]:
print(f'Evaluation of Model1 is {model_1.evaluate(val_inputs,tf.one_hot(val_target,depth=6))}')
print(f'Evaluation of Model2 is {model_2.evaluate(val_inputs,tf.one_hot(val_target,depth=6))}')
print(f'Evaluation of Model3 is {model_3.evaluate(val_inputs,tf.one_hot(val_target,depth=6))}')
print(f'Evaluation of Model4 is {model_4.evaluate(val_inputs,tf.one_hot(val_target,depth=6))}')

3257/3257 ━━━━━━━━━━━━━━━━━━━━ 6s 2ms/step - accuracy: 0.8530 - loss: 0.4232
Evaluation of Model1 is [0.42521438002586365, 0.8516837358474731]
3257/3257 ━━━━━━━━━━━━━━━━━━━━ 7s 2ms/step - accuracy: 0.8564 - loss: 0.4180 - precision: 0.8726 - recall: 0.8486
Evaluation of Model2 is [0.41946345567703247, 0.8554168343544006, 0.8717290759086609, 0.847518801689148]
3257/3257 ━━━━━━━━━━━━━━━━━━━━ 6s 2ms/step - accuracy: 0.8519 - loss: 0.4837 - precision: 0.8646 - recall: 0.8463
Evaluation of Model3 is [0.48614394664764404, 0.8507528305053711, 0.863664448261261, 0.8453307747840881]
3257/3257 ━━━━━━━━━━━━━━━━━━━━ 6s 2ms/step - accuracy: 0.8467 - loss: 0.4896 - precision: 0.8594 - recall: 0.8419
Evaluation of Model4 is [0.4878123998641968, 0.8453019857406616, 0.8579685688018799, 0.8404556512832642]
